In [32]:
import numpy as np

In [9]:
import pandas as pd

In [8]:
import os

In [1]:
from scipy.sparse import csr_matrix, hstack

In [122]:
from sklearn.linear_model import SGDClassifier

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
#For cross validation
from sklearn.cross_validation import StratifiedKFold

In [4]:
#Metric Kaggle evaluates on for this competition
from sklearn.metrics import log_loss

In [5]:
#Specifiying the data directory
datadir = '/Users/Adiii/Documents/Springboard/Capstone Project'

In [10]:
train = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),index_col='device_id')

In [12]:
train.head()

,gender,age,group
device_id,,,
-8076087639492063270,M,35,M32-38
-2897161552818060146,M,35,M32-38
-8260683887967679142,M,35,M32-38
-4938849341048082022,M,30,M29-31
245133531816851882,M,30,M29-31


In [14]:
test = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),index_col='device_id')

In [15]:
test.head()

""
device_id
1002079943728939269
-1547860181818787117
7374582448058474277
-6220210354783429585
-5893464122623104785


In [16]:
#Phone_brand_device_model data
pbd = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))

In [17]:
#There are 529 duplicate device_ids in phone, so we are dropping the duplicates
pbd = pbd.drop_duplicates('device_id',keep='first').set_index('device_id')

In [180]:
pbd.head()

,phone_brand,device_model,brand,model
device_id,,,,
-8890648629457979026,小米,红米,51,858
1277779817574759137,小米,MI 2,51,843
5137427614288105724,三星,Galaxy S4,15,371
3669464369358936369,SUGAR,时尚手机,9,166
-5019277647504317457,三星,Galaxy Note 2,15,347


In [176]:
events = pd.read_csv(os.path.join(datadir,'events.csv'), index_col='event_id')

In [177]:
events.head()

,device_id,timestamp,longitude,latitude
event_id,,,,
1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [178]:
app_events = pd.read_csv(os.path.join(datadir,'app_events.csv'),usecols=['event_id','app_id','is_active'])

In [179]:
app_events.head()

,event_id,app_id,is_active
0,2,5927333115845830913,1
1,2,-5720078949152207372,0
2,2,-1633887856876571208,0
3,2,-653184325010919369,1
4,2,8693964245073640147,1


In [22]:
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))

Feature Engineering:

The features that are going to be used to build our models are phone brand, device model, app labels and the number of apps installed by each device. 

We will be using LabelEncoder to one hot encode to help generate a large number of features. To deal with dimensionality reduction, we will use csr_matrix whose sparse matrices will help us deal with these large number of features. W will apply the same procedure for brand, model,apps installed and app labels.

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
#Encoding different phone brands into numeric values
pb_encoder = LabelEncoder().fit(pbd.phone_brand)

In [26]:
pbd['brand'] = pb_encoder.transform(pbd['phone_brand'])

In [27]:
#Adding the feature brand to train and test set
train['brand'] = pbd['brand']

In [28]:
train.head()

,gender,age,group,brand
device_id,,,,
-8076087639492063270,M,35,M32-38,51
-2897161552818060146,M,35,M32-38,51
-8260683887967679142,M,35,M32-38,51
-4938849341048082022,M,30,M29-31,51
245133531816851882,M,30,M29-31,51


In [29]:
pbd.head()

,phone_brand,device_model,brand
device_id,,,
-8890648629457979026,小米,红米,51
1277779817574759137,小米,MI 2,51
5137427614288105724,三星,Galaxy S4,15
3669464369358936369,SUGAR,时尚手机,9
-5019277647504317457,三星,Galaxy Note 2,15


In [30]:
test['brand']=pbd['brand']

In [182]:
train['trainrow'] = np.arange(train.shape[0])

In [43]:
test['testrow'] = np.arange(test.shape[0])

In [44]:
train_brand = csr_matrix((np.ones(train.shape[0]),(train.trainrow, train.brand)))

In [45]:
train_brand

<74645x131 sparse matrix of type '<class 'numpy.float64'>'
	with 74645 stored elements in Compressed Sparse Row format>

In [46]:
test_brand = csr_matrix((np.ones(test.shape[0]),(test.testrow, test.brand))) 


In [47]:
me = pbd.phone_brand.str.cat(pbd.device_model)

In [48]:
#Encoding different device models into numeric values
mencoder = LabelEncoder().fit(me)

In [49]:
pbd['model'] = mencoder.transform(me)

In [50]:
pbd.head()

,phone_brand,device_model,brand,model
device_id,,,,
-8890648629457979026,小米,红米,51,858
1277779817574759137,小米,MI 2,51,843
5137427614288105724,三星,Galaxy S4,15,371
3669464369358936369,SUGAR,时尚手机,9,166
-5019277647504317457,三星,Galaxy Note 2,15,347


In [183]:
#Adding the feature model to train and test set
train['model']=pbd['model']
train.head()

,gender,age,group,brand,trainrow,model
device_id,,,,,,
-8076087639492063270,M,35,M32-38,51,0,843
-2897161552818060146,M,35,M32-38,51,1,843
-8260683887967679142,M,35,M32-38,51,2,843
-4938849341048082022,M,30,M29-31,51,3,865
245133531816851882,M,30,M29-31,51,4,847


In [52]:
test['model']=pbd['model']

In [53]:
test.head()

,brand,testrow,model
device_id,,,
1002079943728939269,51,0,857
-1547860181818787117,51,1,860
7374582448058474277,31,2,717
-6220210354783429585,31,3,735
-5893464122623104785,51,4,843


In [54]:
train_model = csr_matrix((np.ones(train.shape[0]),(train.trainrow, train.model)))

In [55]:
test_model = csr_matrix((np.ones(test.shape[0]),(test.testrow, test.model))) 

In [187]:
app_encoder = LabelEncoder().fit(app_events.app_id)

In [184]:
app_events['app'] = app_encoder.transform(app_events.app_id)
app_events.head()

,event_id,app_id,is_active,app
0,2,5927333115845830913,1,15408
1,2,-5720078949152207372,0,3384
2,2,-1633887856876571208,0,7620
3,2,-653184325010919369,1,8902
4,2,8693964245073640147,1,18686


We want to know the apps installed for each device. Since the apps are related to devices through events, we merge the Primary Key (device_id) from events to app_events. Perform a groupby to group the result by device_id, app and the size (aggregate). Finally merge it with trainrow and testrow columns.

In [58]:
device_apps=(app_events.merge(events[['device_id']],how='left',left_on='event_id',right_index=True).groupby(['device_id', 'app'])['app'].agg(['size']).merge(train[['trainrow']], how='left', left_index=True, right_index=True).merge(test[['testrow']], how='left', left_index=True, right_index=True).reset_index())

In [59]:
device_apps.head()

,device_id,app,size,trainrow,testrow
0,-9222956879900151005,548,18,21594.0,NaN
1,-9222956879900151005,1096,18,21594.0,NaN
2,-9222956879900151005,1248,26,21594.0,NaN
3,-9222956879900151005,1545,12,21594.0,NaN
4,-9222956879900151005,1664,18,21594.0,NaN


In [60]:
dev = device_apps.dropna(subset=['trainrow'])

In [61]:
n_apps=len(app_encoder.classes_)

In [62]:
train_app = csr_matrix((np.ones(dev.shape[0]), (dev.trainrow, dev.app)),shape=(train.shape[0],n_apps))

In [63]:
dev = device_apps.dropna(subset=['testrow'])

In [64]:
test_app = csr_matrix((np.ones(dev.shape[0]), (dev.testrow, dev.app)),shape=(test.shape[0],n_apps))

Similar to installed apps features above, for app labels we merge applabels with dev (created above). 

In [65]:
applabels = applabels.loc[applabels.app_id.isin(app_events.app_id.unique())]

In [66]:
applabels['app'] = app_encoder.transform(applabels.app_id)

In [67]:
applabels.head()

,app_id,label_id,app
0,7324884708820027918,251,17355
1,-4494216993218550286,251,4618
2,6058196446775239644,406,15548
3,6058196446775239644,407,15548
4,8694625920731541625,406,18689


In [68]:
lencoder = LabelEncoder().fit(applabels.label_id)

In [69]:
applabels['label'] = lencoder.transform(applabels.label_id)

In [70]:
applabels.head()

,app_id,label_id,app,label
0,7324884708820027918,251,17355,207
1,-4494216993218550286,251,4618,207
2,6058196446775239644,406,15548,247
3,6058196446775239644,407,15548,248
4,8694625920731541625,406,18689,247


In [71]:
n_labels = len(lencoder.classes_)

In [72]:
 device_labels=(device_apps[['device_id','app']].merge(applabels[['app','label']]).groupby(['device_id', 'label'])['app'].agg(['size']).merge(train[['trainrow']], how='left', left_index=True, right_index=True).merge(test[['testrow']], how='left', left_index=True, right_index=True).reset_index())

In [73]:
dev = device_labels.dropna(subset=['trainrow'])

In [74]:
train_label = csr_matrix((np.ones(dev.shape[0]), (dev.trainrow, dev.label)),shape=(train.shape[0],n_labels))

In [75]:
dev = device_labels.dropna(subset=['testrow'])

In [76]:
test_label = csr_matrix((np.ones(dev.shape[0]), (dev.testrow, dev.label)),shape=(test.shape[0],n_labels))

We combine all the features created together to create X_train and X_test using hstack.

In [77]:
X_train = hstack((train_brand, train_model, train_app, train_label), format='csr')

In [78]:
X_test = hstack((test_brand, test_model, test_app, test_label), format='csr')

Creating y which consits of the age groups and the gender from the train datatset for predictions.

In [79]:
tencoder = LabelEncoder().fit(train.group)

In [80]:
y = tencoder.transform(train.group)

In [81]:
n = len(tencoder.classes_)

Models: SGD and Logistic Regression

In [138]:
##Stochastic Gradient Descent Classifier
clfsgd = SGDClassifier(loss="log").fit(X_train, y)

In [139]:
clfsgd.predict_proba(X_test)

array([[ 0.00152133,  0.0009339 ,  0.00795414, ...,  0.01077233,
         0.46877925,  0.4150436 ],
       [ 0.00816371,  0.00148126,  0.00953143, ...,  0.00664599,
         0.38459337,  0.21187719],
       [ 0.01927258,  0.00684725,  0.01059325, ...,  0.07019442,
         0.35965636,  0.06875048],
       ..., 
       [ 0.05469399,  0.04355337,  0.04077013, ...,  0.1273684 ,
         0.07303025,  0.06419145],
       [ 0.05469399,  0.04355337,  0.04077013, ...,  0.1273684 ,
         0.07303025,  0.06419145],
       [ 0.05364411,  0.06733343,  0.05225443, ...,  0.08678824,
         0.1161408 ,  0.09775901]])

In [188]:
clfsgd.score(X_train, y)

0.25818206175899255

In [140]:
predsgd = pd.DataFrame(clfsgd.predict_proba(X_test), index = test.index, columns=tencoder.classes_)

In [141]:
predsgd.head()

,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
1002079943728939269,0.001521,0.000934,0.007954,0.006844,0.004904,0.012562,0.002888,0.037019,0.030778,0.010772,0.468779,0.415044
-1547860181818787117,0.008164,0.001481,0.009531,0.062786,0.009079,0.112243,0.000315,0.163081,0.030203,0.006646,0.384593,0.211877
7374582448058474277,0.019273,0.006847,0.010593,0.095274,0.125533,0.093347,0.010207,0.024223,0.116103,0.070194,0.359656,0.068750
-6220210354783429585,0.000516,0.004278,0.000536,0.001468,0.005766,0.111195,0.004507,0.616600,0.041600,0.012550,0.173198,0.027786
-5893464122623104785,0.034650,0.068538,0.039573,0.068462,0.043421,0.035281,0.064706,0.151421,0.097488,0.108561,0.175381,0.112518


In [143]:
##Kaggle score - 2.40891
predsgd.to_csv('pred_sgd2.csv',index=True)

In [157]:
## Logistic Regression
## Performing cross validation
def score(clf1, random_state = 0):
    kf = StratifiedKFold(y, n_folds=2, shuffle=True)
    prediction = np.zeros((y.shape[0],n))
    for itrain, itest in kf:
        Xtrain, Xtest = X_train[itrain, :], X_train[itest, :]
        Ytrain, Ytest = y[itrain], y[itest]
        clf1.fit(Xtrain, Ytrain)
        prediction[itest,:] = clf.predict_proba(Xtest)
        print ("Log loss:", log_loss(Ytest, prediction[itest, :])
               )

In [189]:
clf1.score(X_train, y)

0.24879094380065644

In [171]:
## In logisitic regression, we need to choose C (regularization constant) so that we obtain a good model. 
## The default value of C is 1. I tried different values of C between 0.01-0.1. 
## The log loss was the least for 0.02  
score(LogisticRegression(C=0.02))

Log loss: 2.29060170259
Log loss: 2.29042168821


In [172]:
clf1=(LogisticRegression(C=0.02))
clf1.fit(X_train, y)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [173]:
#Creating a dataframe in the format Kaggle needs for submission
pred1 = pd.DataFrame(clf1.predict_proba(X_test), index = test.index, columns=tencoder.classes_)

In [174]:
##Kaggle score - 2.27331
pred1.to_csv('pred_def_log_reg.csv',index=True)

In [158]:
##Logistic Regression (Multinomial)
def score(clf, random_state = 0):
    kf = StratifiedKFold(y, n_folds=2, shuffle=True)
    prediction = np.zeros((y.shape[0],n))
    for itrain, itest in kf:
        Xtrain, Xtest = X_train[itrain, :], X_train[itest, :]
        Ytrain, Ytest = y[itrain], y[itest]
        clf.fit(Xtrain, Ytrain)
        prediction[itest,:] = clf.predict_proba(Xtest)
        print ("Log loss:", log_loss(Ytest, prediction[itest, :])
               )

Multinomial logistic regression is a classification method that generalizes logistic regression to multiclass problems, i.e. with more than two possible discrete outcomes.

We try a multionomial logistic regression model to optimize logloss. multi_class option could be ovr or multinomial. A binary problem is fit for ovr, whereas for mulitnomial, the loss minimized is the multinomial loss fit across the entire probability distribution. Since this is a multi class problem, we define the solver as lbfgs. Also, multinomial option only works for the lbfgs solver. sag and liblinear are limited to one versus rest schemes.

In [190]:
clf.score(X_train, y)

0.25494004956795496

In [166]:
score(LogisticRegression(C=0.02, multi_class='multinomial',solver='lbfgs'))

Log loss: 2.28123285207
Log loss: 2.28538060975


Yes, this gives us a better log loss. 

In [167]:
clf = LogisticRegression(C=0.02, multi_class='multinomial',solver='lbfgs')
clf.fit(X_train, y)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [168]:
pred = pd.DataFrame(clf.predict_proba(X_test), index = test.index, columns=tencoder.classes_)

In [169]:
pred.head()

,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
1002079943728939269,0.001424,0.005998,0.013605,0.013286,0.025313,0.046103,0.011815,0.033501,0.074615,0.121405,0.260537,0.392400
-1547860181818787117,0.007414,0.013299,0.031228,0.058677,0.072686,0.151391,0.006009,0.091906,0.054989,0.077066,0.229204,0.206130
7374582448058474277,0.023158,0.036713,0.036233,0.158343,0.162774,0.079852,0.012986,0.026441,0.043571,0.104941,0.181501,0.133486
-6220210354783429585,0.003474,0.030860,0.008801,0.012351,0.050697,0.172943,0.048150,0.139719,0.073499,0.093593,0.179861,0.186053
-5893464122623104785,0.046952,0.065640,0.042578,0.062522,0.056329,0.043467,0.090625,0.165219,0.097877,0.102350,0.132938,0.093503


In [170]:
## Kaggle score - 2.26713 
## The best result was obtained from Multinomial Logistic Regression model
pred.to_csv('pred_log_reg.csv',index=True)